# 사용자정의 전처리란?

* 데이터 프레임 또는 시리즈에서 제공하는 기본 함수로 처리하기 어려운 복잡한 전처리를 개발자가 직접 정의하여 처리하는 방식

In [66]:
import pandas as pd
import numpy as np

# Apply 문법

* [데이터프레임 또는 데이터프레임의 열].apply(사용자 정의 함수 또는 람다함수)

In [67]:
data = {
    '상품':['상품1','상품2','상품3','상품4','상품5'],
    '가격($)': [3, 8, 13, 4, 205]}
df = pd.DataFrame(data)
df

,상품,가격($)
0,상품1,3
1,상품2,8
2,상품3,13
3,상품4,4
4,상품5,205


* 사용자 정의 함수 apply 적용

In [68]:
def transform_won(dollar):
    won = dollar * 1395
    return won    

In [69]:
df['가격($)']

0      3
1      8
2     13
3      4
4    205
Name: 가격($), dtype: int64

In [70]:
df['가격($)'].apply( transform_won ) # 데이터프레임의 열에 apply적용하면 사용자 정의 함수에 열의 값이 순차적으로 적용이 된다.

0      4185
1     11160
2     18135
3      5580
4    285975
Name: 가격($), dtype: int64

In [71]:
df['가격($)']

0      3
1      8
2     13
3      4
4    205
Name: 가격($), dtype: int64

In [72]:
df['가격(원)'] =  df['가격($)'].apply(transform_won)
df

,상품,가격($),가격(원)
0,상품1,3,4185
1,상품2,8,11160
2,상품3,13,18135
3,상품4,4,5580
4,상품5,205,285975


* 람다함수 apply 적용  
  간단한 사용자 함수를 정의할 때 사용

In [73]:
df.drop('가격(원)', axis=1, inplace=True)
df

,상품,가격($)
0,상품1,3
1,상품2,8
2,상품3,13
3,상품4,4
4,상품5,205


In [74]:
df['가격(원)'] =  df['가격($)'].apply(lambda dollar: dollar *1395)
df

,상품,가격($),가격(원)
0,상품1,3,4185
1,상품2,8,11160
2,상품3,13,18135
3,상품4,4,5580
4,상품5,205,285975


# 판다스 기본 문법으로 전처리

In [75]:
df.drop('가격(원)', axis=1, inplace=True)
df

,상품,가격($)
0,상품1,3
1,상품2,8
2,상품3,13
3,상품4,4
4,상품5,205


In [76]:
df['가격($)']

0      3
1      8
2     13
3      4
4    205
Name: 가격($), dtype: int64

In [77]:
df['가격($)'] * 1395

0      4185
1     11160
2     18135
3      5580
4    285975
Name: 가격($), dtype: int64

In [78]:
df['가격(원)'] = df['가격($)'] * 1395
df

,상품,가격($),가격(원)
0,상품1,3,4185
1,상품2,8,11160
2,상품3,13,18135
3,상품4,4,5580
4,상품5,205,285975


### 중간 결론  
* 간단한 작업은 굳이 apply 함수를 사용할 필요가 없다.

# 조금 복잡한 경우의 apply 적용

* 등급 열에 대한 요구사항 
    * 한화 가격이 10,000 이하이면 저가상품
    * 한화 가격이 100,000 이하이면 일반상품
    * 한화 가격이 100,000 초과시 고가상품    

In [79]:
df['가격(원)']

0      4185
1     11160
2     18135
3      5580
4    285975
Name: 가격(원), dtype: int64

In [80]:
def get_grade(won_price):
    if won_price <= 10_000:
        return '저가상품'
    elif won_price <= 100_000:
        return '일반상품'
    else:
        return '고가상품'    

In [81]:
df['등급'] = df['가격(원)'].apply(get_grade)
df

,상품,가격($),가격(원),등급
0,상품1,3,4185,저가상품
1,상품2,8,11160,일반상품
2,상품3,13,18135,일반상품
3,상품4,4,5580,저가상품
4,상품5,205,285975,고가상품


In [82]:
df.drop('등급', axis=1, inplace=True)

In [83]:
df

,상품,가격($),가격(원)
0,상품1,3,4185
1,상품2,8,11160
2,상품3,13,18135
3,상품4,4,5580
4,상품5,205,285975


In [84]:
df['등급'] = df['가격(원)'].apply(lambda price: '저가' if price < 10000 else ('일반' if price < 100000 else '고급'))
df

,상품,가격($),가격(원),등급
0,상품1,3,4185,저가
1,상품2,8,11160,일반
2,상품3,13,18135,일반
3,상품4,4,5580,저가
4,상품5,205,285975,고급


## 중간결론

* 복잡한 사용자정의 전처리는 사용자 정의 함수를 apply 메소드에 적용하자!

# 복잡한 경우에서 사용자 전처리

In [139]:
import random

categories = ['의류', '가전제품', '식품', '화장품']

# 상품 데이터 생성
data = []
for index in range(20):
    category = random.choice(categories)
    product = f'상품{index+1}'
    price = random.randint(10, 100)
    data.append([category, product, price])

In [140]:
df = pd.DataFrame(data, columns=['카테고리', '제품','가격($)'])
df.head()

,카테고리,제품,가격($)
0,가전제품,상품1,25
1,의류,상품2,26
2,화장품,상품3,61
3,가전제품,상품4,59
4,식품,상품5,86


### 문제 풀어봅시다.  

* 가격(원) 열을 추가한다.
* 각 카테고리별 할인률을 적용해 봅시다.
* 할인률을 적용한 열 이름을 '할인가격(원)' 으로 한다.
* 카테고리별 할인율은 다음과 같다
    * 의류: 10%
    * 가전제품: 20%
    * 식품: 5%
    * 화장품: 15%

apply 메소드에 사용자 정의 함수를 구현해 풀어보세요!

In [141]:
df['가격(원)'] = df['가격($)'] * 1339
df.head()

,카테고리,제품,가격($),가격(원)
0,가전제품,상품1,25,33475
1,의류,상품2,26,34814
2,화장품,상품3,61,81679
3,가전제품,상품4,59,79001
4,식품,상품5,86,115154


* 데이터프레임.apply(사용자정의함수명 또는 람다함수, axis=1)

In [142]:
def apply_discount(df):
    if df['카테고리'] == '의류':
        return  df['가격(원)'] * 0.9
    elif df['카테고리'] == '가전제품':
        return  df['가격(원)'] * 0.8
    elif df['카테고리'] == '식품':
        return  df['가격(원)'] * 0.95
    elif df['카테고리'] == '화장품':
        return  df['가격(원)'] * 0.85

In [143]:
df['할인가격(원)'] = df.apply( apply_discount, axis = 1)
df

,카테고리,제품,가격($),가격(원),할인가격(원)
0,가전제품,상품1,25,33475,26780.00
1,의류,상품2,26,34814,31332.60
2,화장품,상품3,61,81679,69427.15
3,가전제품,상품4,59,79001,63200.80
4,식품,상품5,86,115154,109396.30
5,의류,상품6,59,79001,71100.90
6,가전제품,상품7,66,88374,70699.20
7,화장품,상품8,29,38831,33006.35
8,화장품,상품9,84,112476,95604.60
9,가전제품,상품10,80,107120,85696.00


# 조금 더 복잡한 케이스

* 위 수행 결과를 리뷰한 결과 아래와 같은 추가 요구사항이 도출이 되었다.
    * 요구사항1] 달러는 앞에 $ 추가한다. 그리고 세자리마다 , 추가한다.
    * 요구사항2] 원화는 금액위에 '원' 표시하고 세자리마다 , 추가
    * 요구사항3] 할인 금액은 정수로 표기
    * 요구사항4] 원본데이터는 숫자로 생성해야 된다.

* 요구사항1

In [91]:
# f'{won:,} => 약식 포맷스트링 문법에서 ':,' 3자리마다 ,를 추가하는 문법
try:
    df['가격(원)'] = df['가격(원)'].apply(lambda won: f'{won:,}')
    df['할인가격(원)'] = df['할인가격(원)'].apply(lambda won: f'{won:,}')
except:
    pass
df.head()

,카테고리,제품,가격($),가격(원),할인가격(원)
0,의류,상품1,41,"54,899","49,409.1"
1,의류,상품2,92,"123,188","110,869.2"
2,화장품,상품3,61,"81,679","69,427.15"
3,의류,상품4,100,"133,900","120,510.0"
4,화장품,상품5,44,"58,916","50,078.6"


* 요구사항2,3

In [146]:
# f'{won:,} => 약식 포맷스트링 문법에서 ':,' 3자리마다 ,를 추가하는 문법
try:
    df['가격(원)'] = df['가격(원)'].apply(lambda won: f'{won:,}원')
    df['할인가격(원)'] = df['할인가격(원)'].apply(lambda won: f'{won:,}원')
except:
    pass
df.head()

,카테고리,제품,가격($),가격(원),할인가격(원)
0,가전제품,상품1,25,"33,475원","26,780원"
1,의류,상품2,26,"34,814원","31,332원"
2,화장품,상품3,61,"81,679원","69,427원"
3,가전제품,상품4,59,"79,001원","63,200원"
4,식품,상품5,86,"115,154원","109,396원"


In [135]:
df.카테고리.head()

0      식품
1      식품
2      의류
3    가전제품
4      식품
Name: 카테고리, dtype: object

In [145]:
df['할인가격(원)'] = df['할인가격(원)'].astype(int)
df['할인가격(원)']

0      26780
1      31332
2      69427
3      63200
4     109396
5      71100
6      70699
7      33006
8      95604
9      85696
10     72506
11     68556
12     53560
13     97613
14     83955
15    106048
16     16871
17    127205
18     91587
19     48204
Name: 할인가격(원), dtype: int64

# 판다스 연습은 아래에서